# pYPK0_PGI_ScXKS1_FBA1

This notebook describes the assembly of the [_Saccaromyces cerevisiae_](www.yeastgenome.org)
single gene expression vector pYPK0_PGI_ScXKS1_FBA1.

It is made by _in-vivo_ homologous recombination between three PCR products and one linear vector fragment.
The PCR products are a promoter generated from a pYPK_Z vector, a gene from a pYPKa_A vector and 
a terminator from a pYPKa_E vector. The three PCR products are joined to
a linearized [pYPKpw](https://github.com/BjornFJohansson/ypk-xylose-pathways/blob/master/pYPKpw.gb) 
backbone vector that has the [URA3](http://www.yeastgenome.org/locus/S000000747/overview) 
marker and a _S. crevisiae_ [2 micron](http://blog.addgene.org/plasmids-101-yeast-vectors) origin of replication. 

The four linear DNA fragments are joined by homologous recombination in a 
[_Saccharomyces cerevisiae_](http://wiki.yeastgenome.org/index.php/Commonly_used_strains) ura3 mutant.

![pYPK0_promoter_gene_terminator](tp_g_tp.png "pYPK0_promoter_gene_terminator")

The [pydna](https://pypi.python.org/pypi/pydna/) package is imported in the code cell below. 
There is a [publication](http://www.biomedcentral.com/1471-2105/16/142) describing pydna as well as
[documentation](http://pydna.readthedocs.org/en/latest/) available online. 
Pydna is developed on [Github](https://github.com/BjornFJohansson/pydna).

In [1]:
import pydna

The YPK [standard primers](standard_primers.txt) are read into a dictionary in the code cell below.

In [2]:
p = { x.id: x for x in pydna.parse("standard_primers.txt") }

The backbone vector [pYPKpw](pYPKpw.gb) is read from a local file in the code cell below.

In [3]:
pYPKpw = pydna.read("pYPKpw.gb")

The backbone vector is linearized by digestion with [EcoRV](http://rebase.neb.com/rebase/enz/EcoRV.html).
The restriction enzyme functionality is provided by [biopython](http://biopython.org).

In [4]:
from Bio.Restriction import EcoRV

pYPK_EcoRV = pYPKpw.linearize(EcoRV)

The pYPKa derived _E. coli_ plasmids containing [promoter](pYPKa_Z_PGI.gb), [gene](pYPKa_A_ScXKS1.gb) and [terminator](pYPKa_E_FBA1.gb)
are read into three variables below.

In [5]:
promoter_template   = pydna.read("pYPKa_Z_PGI.gb")
gene_template       = pydna.read("pYPKa_A_ScXKS1.gb")
terminator_template = pydna.read("pYPKa_E_FBA1.gb")

The construction of the three vector above are described in the [pYPKa_ZE_PGI](pYPKa_ZE_PGI.ipynb) and [pYPKa_A_ScXKS1](pYPKa_A_ScXKS1.ipynb) notebooks.

Three DNA fragments are PCR amplified using [standard primers](standard_primers.txt). Suggested PCR programs can be found at the end of this document.

In [6]:
prom = pydna.pcr( p['577'], p['567'], promoter_template)
gene = pydna.pcr( p['468'], p['467'], gene_template)
term = pydna.pcr( p['568'], p['578'], terminator_template)

The four linear DNA fragments are mixed and transformed
to a _Saccharomyces cerevisiae_ ura3 mutant.

The fragments will be assembled by _in-vivo_ [homologous recombination](http://www.ncbi.nlm.nih.gov/pubmed/2828185):

In [7]:
asm = pydna.Assembly( (pYPK_EcoRV, prom, gene, term), limit=31 )

asm

Assembly:
Sequences........................: [5603] [1231] [1893] [969]
Sequences with shared homologies.: [5603] [1231] [969] [1893]
Homology limit (bp)..............: 31
Number of overlaps...............: 4
Nodes in graph(incl. 5' & 3')....: 6
Only terminal overlaps...........: No
Circular products................: [9224]
Linear products..................: [9468] [9365] [9274] [9261] [8536] [8184] [7418] [6693] [6328] [4006] [3074] [2825] [244] [141] [50] [37]

The representation of the asm object above should normally indicate one circcular product only.  
More than one circular products might indicate an incorrect assembly strategy or represent
by-products that might arise in the assembly process.  
The largest recombination product is chosen as candidate for the pYPK0_PGI_ScXKS1_FBA1 vector.

In [8]:
candidate = asm.circular_products[0]

candidate.figure()

 -|pYPKpw|124
|         \/
|         /\
|         124|1231bp_PCR_prod|50
|                             \/
|                             /\
|                             50|1893bp_PCR_prod|37
|                                                \/
|                                                /\
|                                                37|969bp_PCR_prod|242
|                                                                  \/
|                                                                  /\
|                                                                  242-
|                                                                     |
 ---------------------------------------------------------------------

The candidate vector is synchronized to the backbone vector. This means that
the plasmid origin is shifted so that it matches the pYPKpw backbone vector.

In [9]:
result = candidate.synced(pYPKpw)

### Diagnostic PCR confirmation

The structure of the final vector is confirmed by two
separate PCR reactions, one for the promoter and gene and
one for the gene and terminator.

PCR using standard primers 577 and 467 to amplify promoter and gene.

In [10]:
product = pydna.pcr( p['577'], p['467'], result)

A correct clone should give this size in base pairs:

In [11]:
print(len(product))

3074


If the promoter is missing from the assembly, the PCR product will have this size in base pairs:

In [12]:
print(len(product) - len(prom))

1843


If the gene is missing from the assembly, the PCR product will have this size in base pairs:

In [13]:
print(len(product) - len(gene))

1181


PCR using standard primers 468 and 578 to amplify gene and terminator.

In [14]:
product2 = pydna.pcr( p['468'], p['578'], result)

A correct clone should give this size:

In [15]:
print(len(product2))

2825


If the gene is missing from the assembly, the PCR product will have this size in base pairs:

In [16]:
print(len(product2) - len(gene))

932


If the terminator is missing from the assembly, the PCR product will have this size in base pairs:

In [17]:
print(len(product2) - len(term))

1856


The cseguid checksum for the resulting plasmid is calculated for future reference.
The [cseguid checksum](http://pydna.readthedocs.org/en/latest/pydna.html#pydna.utils.cseguid) 
uniquely identifies a circular double stranded sequence.

In [18]:
result.cseguid()

j4lmfUlfoyUuXDel1WJS3I8-NQE

The file is named based on the nemas of promoter, gene and terminator.

In [19]:
result.locus = "pYPK0_tp_g_tp"
result.definition = "pYPK0_PGI_ScXKS1_FBA1"

Sequence is stamped with cseguid checksum. This can be used to verify the 
integrity of the sequence file.

In [20]:
result.stamp()

cSEGUID_j4lmfUlfoyUuXDel1WJS3I8-NQE_2015-10-02T06:17:23.591000

Write sequence to a local file.

In [21]:
result.write("pYPK0_PGI_ScXKS1_FBA1.gb")

###[pYPK0_PGI_ScXKS1_FBA1](pYPK0_PGI_ScXKS1_FBA1.gb)

## PCR programs for the amplification of Promoter, Gene and Terminator

Promoter

In [22]:
prom.program()


Taq (rate 30 nt/s) 35 cycles             |1231bp
95.0°C    |95.0°C                 |      |SantaLucia 1998
|_________|_____          72.0°C  |72.0°C|SaltC 50mM
| 03min00s|30s  \         ________|______|
|         |      \ 56.0°C/ 0min36s| 5min |
|         |       \_____/         |      |
|         |         30s           |      |4-12°C

Gene

In [23]:
gene.program()


Taq (rate 30 nt/s) 35 cycles             |1893bp
95.0°C    |95.0°C                 |      |SantaLucia 1998
|_________|_____          72.0°C  |72.0°C|SaltC 50mM
| 03min00s|30s  \         ________|______|
|         |      \ 57.0°C/ 0min56s| 5min |
|         |       \_____/         |      |
|         |         30s           |      |4-12°C

Terminator

In [24]:
term.program()


Taq (rate 30 nt/s) 35 cycles             |969bp
95.0°C    |95.0°C                 |      |SantaLucia 1998
|_________|_____          72.0°C  |72.0°C|SaltC 50mM
| 03min00s|30s  \         ________|______|
|         |      \ 56.0°C/ 0min29s| 5min |
|         |       \_____/         |      |
|         |         30s           |      |4-12°C

## Download [pYPK0_PGI_ScXKS1_FBA1](pYPK0_PGI_ScXKS1_FBA1.gb)

In [25]:
import pydna
reloaded = pydna.read("pYPK0_PGI_ScXKS1_FBA1.gb")
reloaded.verify_stamp()

cSEGUID_j4lmfUlfoyUuXDel1WJS3I8-NQE